<!-- # **Near Real-Time Probabilistic Plasma Flow Analysis using LSTM and DSCOVR Data from the Lagrangian Point (L1)** -->
# **Near Real-Time Probabilistic Plasma Flow Analysis using LSTM, Dense Layers and DSCOVR Data from the Lagrangian point (L1)**

[DSCOVR](https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.ngdc.stp.swx:satellite-systems_dscovr) continues to operate beyond its expected lifespan, occasionally producing hardware faults that may result from space weather events. In this notebook, we analyze raw data from NASA's [DSCOVR](https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.ngdc.stp.swx:satellite-systems_dscovr) mission to forecast geomagnetic storms, taking into account hardware faults.

For this project, we draw upon prior research, specifically:

- [Cristoforetti, M., Battiston, R., Gobbi, A. et al. “Prominence of the training data preparation in geomagnetic storm prediction using deep neural networks.“](https://www.nature.com/articles/s41598-022-11721-8)  Sci Rep 12, 7631 (2022).
- [Marina Stepanova et al. “Prediction of Geomagnetic Storm Using Neural Networks:Comparison of the Efficiency of the Satellite and GroundBased Input Parameters.“](https://iopscience.iop.org/article/10.1088/1742-6596/134/1/012041/pdf) 2008 J. Phys.: Conf. Ser. 134 012041
- [Tasistro-Hart, Adrian et al. “Probabilistic Geomagnetic Storm Forecasting via Deep Learning.”](https://www.semanticscholar.org/paper/Probabilistic-Geomagnetic-Storm-Forecasting-via-Tasistro-Hart-Grayver/45103139959e86620de7353615db70e4e730efe5)  Journal of Geophysical Research: Space Physics 126 (2020): n. pag.
- [Gulati, Ishita et al. “Classification based Detection of Geomagnetic Storms using LSTM Neural Network.”](https://www.semanticscholar.org/paper/Classification-based-Detection-of-Geomagnetic-using-Gulati-Li/96d8ff1d64f3d2691ee42e1b3ea6c33cc4136d6f) 2022 3rd URSI Atlantic and Asia Pacific Radio Science Meeting (AT-AP-RASC) (2022): 1-4.
- [Smith, A. W., Forsyth, C., Rae, I. J., Garton, T. M., Jackman, C. M., Bakrania, M., et al. On the considerations of using near real time data for space
weather hazard forecasting.](https://spiral.imperial.ac.uk/bitstream/10044/1/98383/2/Smith_SpaceWeather_20_e2022SW003098_2022.pdf) Space Weather, 20, (2022) e2022SW003098. https://doi.org/10.1029/2022SW003098

Our objective is to implement the model in a real-time or near real-time environment, enabling us to process [DSCOVR](https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.ngdc.stp.swx:satellite-systems_dscovr) data and make predictions.

## **Understanding Solar Wind**

Solar wind frequently interacts with Earth's magnetosphere, leading to geomagnetic storms that can disrupt various technologies, including satellites and electrical power grids. 
To address this challenge, the National Oceanic and Atmospheric Administration (NOAA) operates a space weather station known as the Deep Space Climate Observatory (DSCOVR). 

[DSCOVR](https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.ngdc.stp.swx:satellite-systems_dscovr) is equipped with a range of sensors designed to predict these storms by collecting vital data on the speed, temperature, and density of incoming solar plasma.

[DSCOVR](https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.ngdc.stp.swx:satellite-systems_dscovr) maintains a orbit, stationed at Lagrange point one, positioned 1.5 million kilometers from Earth and situated between our planet and the Sun. NOAA uses this data to simulate Earth's magnetic field and atmosphere, offering potential early warnings of geomagnetic storms.


Geomagnetic activity is commonly quantified using the $D_{st}$ index. Using solar wind parameters and magnetic field data, we can potentially predict changes in the $D_{st}$ index. Prior studies recommend incorporating interplanetary parameters as inputs [[1]](https://www.nature.com/articles/s41598-022-11721-8), including the interplanetary magnetic field ($IMF$), solar wind ($SW$), and in some cases, specific components like the IMF $B_z$, $SW$ electric field, temperature, speed, and density.

It is worth noting that the study by Tasistro-Hart et al. incorporates observations from the solar disk as additional input data, suggesting that this inclusion enhances forecasting reliability. Furthermore, they shift their focus from predicting $D_{st}$ to the external component of geomagnetic storms, known as $E_{st}$, and their neural network architecture is adept at forecasting uncertainty.

Note that,

$D_{st} = I_{st}(t) + E_{st}(t)$

In this notebook, we build upon prior research in the field, specifically the work outlined in [[2]](https://www.semanticscholar.org/paper/Probabilistic-Geomagnetic-Storm-Forecasting-via-Tasistro-Hart-Grayver/45103139959e86620de7353615db70e4e730efe5). 

Our approach involves implementing a machine learning model while accounting for the impact of hardware faults and anomalies. Finally, our goal is to provide uncertainty estimates for the output, following the methodology described by Tasistro-Hart et al.

However, it's important to note that meticulous data preparation for training and validation significantly influence the performance of the machine learning model.

### **Data Resources**

For this project, we will directly utilize raw [DSCOVR](https://www.ncei.noaa.gov/access/metadata/landing-page/bin/iso?id=gov.noaa.ngdc.stp.swx:satellite-systems_dscovr) data as input. Given our objective of building a near-real-time system, we have opted to integrate the most recent data by implementing a function that can download datasets from the [Experimental Data Repository](https://www.spaceappschallenge.org/develop-the-oracle-of-dscovr-experimental-data-repository/).

These datasets encompass data starting from 2016 and continue to receive updates to the present day. They will be stored in the `dataset` folder. Each `.csv` file contains 53 columns, with column 0 representing the time in UTC in the following format:  `YYYY-MM-DD hh:mm:ss`. Columns 1-3 correspond to the magnetic field components measured in nanoteslas (nT) at the time indicated in column 0. The remaining columns contain dimensional measurements from the Faraday cup plasma detector.

#### **References**
[1] [Prominence of the training data preparation in geomagnetic storm prediction using deep neural networks](https://www.nature.com/articles/s41598-022-11721-8)

[2] [Probabilistic Geomagnetic Storm Forecasting via Deep Learning](https://www.semanticscholar.org/paper/Probabilistic-Geomagnetic-Storm-Forecasting-via-Tasistro-Hart-Grayver/45103139959e86620de7353615db70e4e730efe5)
- [DSCOVR: Deep Space Climate Observatory](https://www.nesdis.noaa.gov/current-satellite-missions/currently-flying/dscovr-deep-space-climate-observatory)
- [DSCOVR (Deep Space Climate Observatory) -eoPortal](https://www.eoportal.org/satellite-missions/dscovr)
- [Deep Space Climate Observatory (DSCOVR)](https://www.nist.gov/measuring-cosmos/deep-space-climate-observatory-dscovr)


## **Data Retrieval and Preprocessing**

For this prototype, we utilise the experimental data provided during the competition. It is worth noting that in a real-time production environment, the functionality of the following function can be extended to facilitate the real-time acquisition and storage of data in a MongoDB database for efficient retrieval and processing.

A virtual environment was created and the required libraries were installed to develop this project. Navigate to the project folder and type the following:

```
python -m venv ./venv

pip install -r requirements.txt

``` 

To activate the virtual environment please type:

```
source ./venv/bin/activate
```

In [7]:
# importing required libraries for data retrival and pro-processing

import numpy as np
import pandas as pd
import matplotlib as mtp

In [9]:
# check versions

print("Pandas: {}".format(pd.__version__))
print("Numpy: {}".format(np.__version__))
print("matplotlib: {}".format(mtp.__version__))

Pandas: 2.1.1
Numpy: 1.26.0
matplotlib: 3.8.0


In [31]:
import datetime as dt
from tqdm import tqdm
import os
import zipfile
import requests

root_url = "https://opensource.gsfc.nasa.gov/spaceappschallenge/"
# The data start being recorded from 2016; DSCOVR became operational on July 27, 2016.
start_year = 2016
today = dt.date.today()
current_year = today.year

dataset_folder = "../dataset"  # Dataset forlder to store .csv files/unzipped data
tmp_folder = "../tmp"  # Temporary folder to store downloaded .zip files
os.makedirs(tmp_folder, exist_ok=True)
os.makedirs(dataset_folder, exist_ok=True)


def fetch_experimental_dscovr_data():
    """Download and store experimental DSCOVR data.

    The data is stored in .zip folders, which are first downloaded to a tmp folder,
    and then extracted into the dataset folder before the zip files are erased.

    Note: If the file is from the current year, it is always downloaded, even if it already exists.
    """
    for year in range(start_year, current_year + 1):
        url = root_url + "dsc_fc_summed_spectra_{}_v01.zip".format(year)
        zip_filename = os.path.join(tmp_folder, "dscovr_data_{}.zip".format(year))
        dataset_year_folder = os.path.join(dataset_folder, str(year))

        # Remove existing files for the current year (if they exist)
        if year == current_year:
            existing_files = [
                f
                for f in os.listdir(dataset_year_folder)
                if f.startswith(f"dsc_fc_summed_spectra_{year}")
            ]
            for existing_file in existing_files:
                os.remove(os.path.join(dataset_year_folder, existing_file))

        # Check if the file already exists, if not, download it
        if not any(
            f.startswith(f"dsc_fc_summed_spectra_{year}")
            for f in os.listdir(dataset_year_folder)
        ):
            try:
                # Download the zip file
                print("Downloading DSCOVR data for year {}...".format(year))
                response = requests.get(url, stream=True)
                total_size = int(response.headers.get("content-length", 0))

                if response.status_code == 200:
                    # Display a progress bar when downloading experimental DSCOVR dataset
                    with open(zip_filename, "wb") as file, tqdm(
                        desc=zip_filename,
                        total=total_size,
                        unit="B",
                        unit_scale=True,
                        unit_divisor=1024,
                    ) as bar:
                        for data in response.iter_content(chunk_size=1024):
                            file.write(data)
                            bar.update(len(data))
                    print("Download complete for year {}.".format(year))

                    # Unzip the downloaded file into the dataset folder
                    with zipfile.ZipFile(zip_filename, "r") as zip_ref:
                        zip_ref.extractall(dataset_year_folder)

                    # Delete the downloaded zip file from the temporary folder
                    os.remove(zip_filename)
                else:
                    print("Failed to download data for year {}.".format(year))
            except Exception as e:
                print(
                    "An error occurred while downloading DSCOVR data for year {}: {}".format(
                        year, e
                    )
                )
        else:
            print("DSCOVR Data for year {} already exists.".format(year))


def prepare_experimental_Kp_data():
    dataset_folder = '../dataset/'
    kp_folder = 'kp_data'
    



def prapare_dataframe():
    """Loop through the .csv files to create a Pandas DataFrame.

    This function reads CSV files from the dataset folder for the specified range of years
    and combines them into a single Pandas DataFrame. It handles missing files gracefully.

    Returns:
        pandas.DataFrame: A combined DataFrame containing data from all available CSV files.
    """

    combined_df = pd.DataFrame()
    dataset_folder = '../dataset/'
    for year in range(start_year, current_year + 1):
        path_csv_file = os.path.join(dataset_folder, str(year), f"dsc_fc_summed_spectra_{year}_v01.csv")
        # Check if the file exists before trying to read it
        if os.path.exists(path_csv_file):
            df = pd.read_csv(
                path_csv_file,
                delimiter=",",
                parse_dates=[0],
                na_values="0",
                header=None,
            )
            combined_df = pd.concat([combined_df, df])
        else:
            print(f"Warning: File {path_csv_file} does not exist.")
            
    return combined_df



In [32]:
# Call the functions to download data and create a dataframe

fetch_experimental_dscovr_data()

df = prapare_dataframe()


## **Exploratory Data Analysis (EDA)**

In this section, we employ Pandas and data visualization libraries to gain a deeper understanding of the experimental DSCOVR data. Our aim is to identify patterns that will inform the selection of the most suitable machine learning model for our problem statement.

We begin by providing a high-level overview of the data. Subsequently, we proceed with data cleaning, addressing issues such as missing values. Next, we delve into the exploration of correlations between variables, presenting our findings through visual representations.  Finally, we summarise the key findings from this section.

### **Data Summary**

Following is a high level overview of the data. Additional information on how to interpret the columns can be found [here](https://hpde.io/NASA/NumericalData/OMNI/PT1H)

In [33]:
df.head()

# A note regarding column 4-53: 5th+ columns: These are the "raw" measurements of the spectrum from the plasma detector so they are the "level 1" data. 
# We're reading it as each value is the flow strength of the solar wind, with each column being an interval in a range of flow speeds at which the solar wind is traveling at.

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
0,2016-01-01 00:00:00,6.83609,-3.37934,-12.9205,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2016-01-01 00:01:00,6.76732,-3.30194,-12.9967,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2016-01-01 00:02:00,6.39107,-2.61173,-13.3271,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2016-01-01 00:03:00,6.44897,-2.61525,-13.3299,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2016-01-01 00:04:00,6.58758,-2.73082,-13.2361,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [34]:
# providing a concise summary of a dataframe.
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3277440 entries, 0 to 175679
Data columns (total 54 columns):
 #   Column  Dtype         
---  ------  -----         
 0   0       datetime64[ns]
 1   1       float64       
 2   2       float64       
 3   3       float64       
 4   4       float64       
 5   5       float64       
 6   6       float64       
 7   7       float64       
 8   8       float64       
 9   9       float64       
 10  10      float64       
 11  11      float64       
 12  12      float64       
 13  13      float64       
 14  14      float64       
 15  15      float64       
 16  16      float64       
 17  17      float64       
 18  18      float64       
 19  19      float64       
 20  20      float64       
 21  21      float64       
 22  22      float64       
 23  23      float64       
 24  24      float64       
 25  25      float64       
 26  26      float64       
 27  27      float64       
 28  28      float64       
 29  29      float64     

In [37]:
# return the number of rows
len(df.index)

3277440

In [35]:
# Basic descriptive statistics for all columns
df.describe(include='all')

,0,1,2,3,4,5,6,7,8,9,...,44,45,46,47,48,49,50,51,52,53
count,3277440,3.259998e+06,3.259998e+06,3.259998e+06,1.519804e+06,1.582311e+06,1.721032e+06,1.809991e+06,1.922099e+06,2.008476e+06,...,179435.000000,131650.000000,103457.000000,70114.000000,61407.000000,37701.000000,34288.000000,29597.000000,28458.000000,26270.000000
mean,2019-11-24 16:42:18.717038592,8.015442e-02,-1.255092e-01,2.171055e-02,6.904954e+01,2.405151e+01,9.262740e+01,9.369958e+01,1.246801e+02,1.251655e+02,...,410.258331,353.559684,397.477883,381.584273,351.028206,364.846009,364.814574,339.140779,387.293616,339.150626
min,2016-01-01 00:00:00,-2.014280e+01,-3.178510e+01,-3.332440e+01,2.317260e-01,2.317260e-01,2.317260e-01,2.317260e-01,2.317260e-01,2.317260e-01,...,96.621000,63.826300,63.398300,2.675280,2.947210,59.301300,76.164100,65.362400,0.231726,2.469710
25%,2018-01-04 11:59:45,-2.532720e+00,-2.640140e+00,-1.522248e+00,3.139748e+01,2.317260e-01,4.230860e+01,3.983035e+01,5.333525e+01,4.449867e+01,...,392.940000,336.209750,387.257000,372.588500,339.152500,365.221000,370.332750,341.495000,397.712750,343.118500
50%,2020-03-20 23:59:30,1.694915e-01,-2.131885e-01,3.084400e-02,5.893845e+01,1.063320e+01,8.518255e+01,8.183820e+01,1.085690e+02,1.006845e+02,...,421.542000,367.358000,416.448000,409.860500,380.525000,397.663000,397.949500,373.665000,431.640000,381.959500
75%,2021-10-10 23:59:15,2.672278e+00,2.395460e+00,1.553040e+00,9.426612e+01,3.352490e+01,1.191900e+02,1.199030e+02,1.635685e+02,1.546990e+02,...,437.436000,385.025750,432.293000,423.639000,397.720000,418.133000,419.557750,394.069000,451.927750,401.169250
max,2023-05-02 23:59:00,3.304940e+01,2.789380e+01,3.483770e+01,1.675760e+03,1.582720e+03,1.736050e+03,1.496590e+03,1.699290e+03,1.848460e+03,...,1756.870000,1757.440000,1775.960000,1762.550000,1689.330000,1719.110000,1939.020000,1852.740000,1875.050000,1866.960000
std,NaN,3.410268e+00,3.765791e+00,2.982616e+00,6.505515e+01,4.948382e+01,7.784311e+01,8.532521e+01,1.040021e+02,1.253738e+02,...,66.313573,66.450856,72.205229,84.113211,85.454188,99.350766,104.976669,109.746891,118.891957,112.803118


In [40]:
# return the list of column names of the dataframe
df.columns


Index([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
       36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53],
      dtype='int64')

### **Data Cleaning**

In this section, our focus is on data cleaning, which involves addressing missing values and outliers early in the process. Missing values can adversely affect the accuracy of a machine learning model. There are a couple of options for handling them: you can either drop rows or columns with missing values, or you can fill in the missing values with the median, mean, or mode. To determine the best approach, you can count the number of `NaN` values in the dataframe and also understand the significance of these missing valies.

In [41]:
# count the number of missing values
df.isnull().sum()
# we have 3277440 rows

0           0
1       17442
2       17442
3       17442
4     1757636
5     1695129
6     1556408
7     1467449
8     1355341
9     1268964
10    1148804
11    1053137
12     921593
13     853886
14     732934
15     682399
16     623358
17     554684
18     506961
19     472456
20     425829
21     425497
22     434438
23     521356
24     573915
25     749402
26     846987
27    1001870
28    1124509
29    1315618
30    1457869
31    1690322
32    1784844
33    2032300
34    2114747
35    2218250
36    2396398
37    2499624
38    2570894
39    2726544
40    2775096
41    2936463
42    2979187
43    3035621
44    3098005
45    3145790
46    3173983
47    3207326
48    3216033
49    3239739
50    3243152
51    3247843
52    3248982
53    3251170
dtype: int64

In a paper by Smith, A. W., Forsyth, C., Rae, I. J., Garton, T. M., Jackman, C. M., Bakrania, M., et al. (2022), it is noted that despite DSCOVR data being collected in real-time, there are instances where data from ACE are used to fill in missing data for larger intervals. Given the short duration of this project (2 days), we have chosen to employ limited interpolation to address missing values for gaps of 5 minutes or less, as suggested in the paper.

Due to time constraints, we will use this method. There are different types of interpolation that can be applied to time series data, including:
- linear interpolation (very common)
- Polynomial interpolatin
- Spline interpolation
- Time-Based Interpolation
- Exponential Smoothing
- Seasonal Decomposition

For this project we used linear interpolation for columns 4-53.

In [ ]:
def linear_interpolation(df:  pd.DataFrame, init_column: int, end_column: int, gap: int):
    """
    Linearly interpolate missing values in specified columns of a DataFrame.

    Parameters:
    df (pd.DataFrame): The DataFrame containing the data to be interpolated.
    init_column (int): The index of the first column to start interpolation.
    end_column (int): The index of the last column to end interpolation (exclusive).
    gap (int): The maximum gap size (in minutes) to be filled with interpolation.

    Returns:
    pd.DataFrame: The DataFrame with missing values interpolated.
    """
    # Select the columns to be interpolated
    columns_to_interpolate = df.columns[init_column:end_column]
    # Apply linear interpolation to the specified columns
    df[columns_to_interpolate] = df[columns_to_interpolate].interpolate(method='linear', limit=gap)

    return df


## **Feature Engineering** 

We also undertake feature engineering to create new attributes necessary for training our machine learning model.